# Install dependencies
$ conda install -c conda-forge opencv
$ conda install -c conda-forge Pillow
$ conda install flask
$ conda install -c conda-forge numpy
$ conda install -c conda-forge flask-socketio
$ conda install -c conda-forge eventlet
$ conda install -c conda-forge keras

In [1]:
import argparse
import base64
import json
import numpy as np
import time
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

# Switch Keras to TensorFlow
import tensorflow as tf
tf.python.control_flow_ops = tf

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

Using Theano backend.


In [2]:
# $ conda install eventlet
import eventlet
import eventlet.wsgi

ImportError: No module named 'eventlet'

In [ ]:
#$ anaconda search -t conda socketio
import socketio

In [ ]:
sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None

@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_array = np.asarray(image)
    transformed_image_array = image_array[None, :, :, :]
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = float(model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = 0.2
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)




In [ ]:
@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument('model', type=str,
    help='Path to model definition h5. Model should be on the same path.')
    args = parser.parse_args()

    model = load_model(args.model)

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)
Server
computing A program that provides services to other programs or users, either in the same computer or over a computer network.
computing A computer dedicated to running such programs.
One who serves; a waitress or waiter.